In [2]:
%matplotlib inline

import warnings
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels as sm
import matplotlib
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
dt = pd.read_csv('total_waterborne_commerce.csv')
dt.head()

In [ ]:
# fake1 = [1990 for i in range(0, 40)]
fake1 = np.linspace(0, 40, 40, dtype=int)
# fake1
# replace_at = 
# np.random.random_integers(1,5)
print(fake1)
replace_at = np.random.randint(0, 40, size=10)
for i in range(0, len(fake1)):
    if(i in replace_at):
        fake1[i] = np.random.randint(0, 40)
print(fake1)

In [ ]:
col1 = dt['Year'].values
col2 = dt['Total'].values
col3 = dt['Foreign'].values
col4 = dt['Domestic'].values
col4.ravel()

In [ ]:
# Create models from data
def best_fit_distribution(data, bins=200, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    # Distributions to check
    DISTRIBUTIONS = [st.uniform, st.norm, st.zipf, st.linregress]
#     DISTRIBUTIONS = [st.linregress]

    # Best holders
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf

    # Estimate distribution parameters from data
    for distribution in DISTRIBUTIONS:

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')

                # fit dist to data
                params = distribution.fit(data) #TODO Study return parameters for different distributions

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]

                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))

                # if axis pass in add to plot
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                    end
                except Exception:
                    pass

                # identify if this distribution is better
                if best_sse > sse > 0:
                    best_distribution = distribution
                    best_params = params
                    best_sse = sse

        except Exception:
            pass

    return (best_distribution.name, best_params)

In [ ]:
# Load data from statsmodels datasets
# data = pd.Series(sm.datasets.elnino.load_pandas().data.set_index('YEAR').values.ravel())
# data = pd.Series(fake1)
data = pd.Series(col1)

# Plot for comparison
plt.figure(figsize=(12,8))
ax = data.plot(kind='hist', bins=50, normed=True, alpha=0.5, color=plt.rcParams['axes.color_cycle'][1])
# Save plot limits
dataYLim = ax.get_ylim()

# Find best fit distribution
best_fit_name, best_fir_paramms = best_fit_distribution(data, 200, ax)
best_dist = getattr(st, best_fit_name)
print("{0} , {1}, {2}".format(best_fit_name, best_fir_paramms, best_dist ))

In [ ]:
best_dist